In [1]:
OPENAI_API_KEY=""

In [2]:
from pydantic import BaseModel, Field
from typing import Literal
from openai import OpenAI
import instructor
from raphtory import PersistentGraph

In [3]:
def expand_from_subgraph(nodes, limit=10):
    return nodes + [neighbour.name for node in nodes for neighbour in list(g.node(node).neighbours)[0:limit]]

g = PersistentGraph.load_from_file("/tmp/aml/graphs/master_graph")
    
reports = [node.name for node in g.nodes if node.properties.get("type") == "Report"]
reports_and_related = expand_from_subgraph(reports)

suspicious = ["Polux Management Lp", "Lcm Alliance Llp", "Riverlane Llp", "Larkstone Limited"]
suspicious_and_related = expand_from_subgraph(suspicious)

g = g.subgraph(reports_and_related + suspicious_and_related).materialize()

In [22]:
def get_embeddings(strings, model="text-embedding-3-small"):
    client = OpenAI(api_key=OPENAI_API_KEY)
    return [client.embeddings.create(input=text, model=model).data[0].embedding for text in strings]

node_document = """
{% if properties.type == "Company" %}
{{ name }} is a company with the following details:
Employee count: {{ properties.employeeCount}}
Account: {{ properties.account}}
Location: {{ properties.location}}
Jurisdiction: {{ properties.jurisdiction}}
Partnerships: {{ properties.partnerships}}
{% endif %}
{% if properties.type == "Person" %}
{{ name }} is a director with the follwing details:
Age: {{ properties.age }}
Mobile: {{ properties.mobile }}
Home address: {{ properties.homeAddress }}
Email: {{ properties.email }}
{% endif %}
{% if properties.type == "Report" %}
{{name}} is a suspicious activity report with the following content:
{{ properties.document }}
{% endif %}
"""

edge_document = """
{% if layers[0] == "report" %}
{{ src.name }} was raised against {{ dst.name}}
{% elif layers[0] == "director" %}
{{ dst.name }} is a director of {{ src.name }}
{% else %}
{{ src.name }} transferred ${{ properties.amount_usd }} to {{ dst.name }}
{% endif %}
"""

v = g.vectorise(get_embeddings, nodes=node_document, edges=edge_document, verbose=True)


Hilux Services Lp sent $2000000 to Geys Sirajli



In [34]:
client = instructor.from_openai(OpenAI(api_key=OPENAI_API_KEY))

def send_query_with_docs(query: str, selection):
    formatted_docs = "\n".join(doc.content for doc in selection.get_documents())
    instructions = f"You are helpful assistant. Answer the user question using the following context:\n{formatted_docs}"
    response = client.responses.create(
        model="o4-mini",
        instructions=instructions,
        input=query,
    )
    print(response.output_text)

def default_pipeline(query: str):
    print(">>> Using the DEFAULT pipeline")
    
    # We just use regular similarity search plus expansion by similarity in the default pipeline
    s = v.entities_by_similarity(query, limit=10)
    s.expand_entities_by_similarity(query, limit=10)
    
    send_query_with_docs(query, s)

def aml_pipeline(query: str):
    print(">>> Using the AML pipeline")
    
    # We start by including all the suspicious activity reports in this case
    reports = [node for node in g.nodes if node.properties.get("type") == "Report"]
    s = v.empty_selection()
    s.add_nodes(reports)
    s.expand_entities_by_similarity(query, limit=10)

    # We make sure to include the largest money transfers in this subset of the network
    additional_edges = [edge for node in s.nodes() for edge in node.edges]
    largest_transfers = sorted(additional_edges, key=lambda edge: edge.properties.get("amount_usd"))[-10:]
    transfer_parties = [node for edge in largest_transfers for node in [edge.src, edge.dst]]
    s.add_edges([(edge.src, edge.dst) for edge in largest_transfers])
    s.add_nodes(transfer_parties)
    
    send_query_with_docs(query, s)

def kyc_pipeline(query: str):
    print(">>> Using the KYC pipeline")
    
    # In this case we know we are targeting a node, that might be a person or a company:
    s = v.nodes_by_similarity(query, limit=3)
    
    # then instead of expanding by similarity we include all the context
    s.expand(hops=2)
    
    send_query_with_docs(query, s)

class ClassificationResponse(BaseModel):
    """
    A few-shot example of agent classification:

    Examples:
    - "Write a KYC report for Metastar Invest Llp": KYC
    - "Can you find any activities suspicious of money laundry for Esmira Jamalkhanova?": AML
    - "Tell me what you know about Larkstone Limited": DEFAULT
    """
    agent: Literal["KYC", "AML", "DEFAULT"] = Field(
        ...,
        description="The agent to use to answer the question. KYC when the user asks for some KYC report. AML when the user wants to identify money landry patterns or suspicious activity. Otherwise, use the agent DEFAULT",
    )

def pipeline(query: str):
    response = client.chat.completions.create(
        model="o4-mini",
        response_model=ClassificationResponse,
        messages=[{"role": "user", "content": query}]
    )
    if response.agent == "AML":
        aml_pipeline(query)
    elif response.agent == "KYC":
        kyc_pipeline(query)
    else:
        default_pipeline(query)

In [28]:
# AML example
pipeline("Can you find any suspicious activity")

>>> Using the AML pipeline
5
6
Based on the information and SARs you’ve provided, the following stand out as suspicious:

1. Hilux Services Lp → Geys Sirajli  
   – Amount: USD 2,000,000  
   – Hilux Services Lp is a UK-jurisdiction LLP with no employees or partnerships (i.e. likely a shell).  
   – Geys Sirajli (director at Stellar Innovations) is already the subject of SAR 1 for “unexplained wealth…potential Russian Laundromat links.”  
   – A single, large inbound payment like this is inconsistent with his known business activities and flagged business rationale.

2. Metastar Invest Lp → Esmira Jamalkhanova  
   – Amount: EUR 248,884  
   – Metastar Invest Llp is also a UK LLP with no employees or partnerships (another potential shell).  
   – Esmira Jamalkhanova (director at Nexus Electronics) is subject of SAR 2 for “transactions involving sanctioned entities…links to organized crime.”  
   – A sudden, sizeable credit from a shell-type entity fits the pattern of her flagged behavi

In [32]:
pipeline("How much money has Hilux Services Lp sent to Geys Sirajli")

>>> Using the DEFAULT pipeline
Hilux Services Lp sent Geys Sirajli 2,000,000 USD.


In [35]:
pipeline("Please write a KYC report about Hilux Services Lp")

>>> Using the KYC pipeline
KYC REPORT: HILUX SERVICES LP  
Date: 2025-07-11  

1. Company Identity  
• Legal Name: Hilux Services Lp  
• Jurisdiction: United Kingdom  
• Bank Account: EE12 3300 3335 1615 0001  
• Employee Count: None declared  
• Partnerships/Affiliations: None declared  

2. Ownership & Management  
• No directors or ultimate beneficial owners (UBOs) are on record.  
• No employees or local footprint to support declared transactional volumes—ownership and governance information is incomplete.  

3. Business Profile & Purpose  
• No stated business activity, address or operating premises.  
• No trade or service description has been provided to justify large cross-border flows.  

4. Transaction Overview (Jan–Jun 2025)  
Total Outflows: ~USD 5.7 million + EUR 1.9 million + CHF 53,000+  
Total Inflows: ~USD 1.4 million + EUR 170,000+  

Key counterparty flows:  
• Polux Management Lp (UK): Outflows USD 673,000; multiple EUR/USD/CHF transfers; round-trip: Hilux→Polux→Riv

In [31]:
pipeline("tell me everything you know about Metastar Invest Llp")

>>> Using the DEFAULT pipeline
Here’s a consolidated profile of Metastar Invest LLP based on the information provided:

1. Basic Company Details  
   • Legal form: Limited Liability Partnership (LLP)  
   • Jurisdiction: United Kingdom  
   • Number of employees: None recorded  
   • Registered location: Not specified  
   • Known partnerships: None recorded  
   • Bank account (IBAN): EE77 3300 3334 8704 0004  

2. Recorded Fund Transfers  
   A. Outgoing (Metastar Invest LLP → …)  
     – 1,200,000 USD → Metastar Invest LLP (internal/self-transfer)  
     – 63 EUR → Larkstone Limited  
     – 100,300 USD → LCM Alliance LLP  
     – 248,884 EUR → Esmira Jamalkhanova  
     – 29,700 USD → KG Commerce LLP  
     – 52,627 EUR → Eduard Lintner  

   B. Incoming (… → Metastar Invest LLP)  
     – 634,372 USD ← Larkstone Limited  
     – 56,090 USD ← KG Commerce LLP  
     – 177,332 USD ← LCM Alliance LLP  

3. Summary of Transaction Volumes  
   • Total USD outflow: 1,330,000 USD (includin